In [1]:
import nltk
import re
import Hangul

# data는 동일한 directory에 위치해 있다.

# file에서 data를 받아온다.
with open("train.txt", "r") as f1:
    train_data = f1.readlines()

with open("test.txt", "r") as f2:
    test_data = f2.readlines()

# 받은 data를 \t, \n (whitespace) 을 기준으로 나눈다.
for i in range(len(train_data)):
    train_data[i] = train_data[i].split()
    
for i in range(len(test_data)):
    test_data[i] = test_data[i].split()

In [18]:
# 필요한 dataset 구현
cvl_data_set = []
for data in train_data:
    if data[2] == 'CVL':
        if re.search("\w+", data[1]):
            word = re.search("\w+", data[1]).group()
            if word[-1] in "은는이가을던의를에도들":
                cvl_data_set.append(word[:-1])
            else:
                cvl_data_set.append(word)
                
CVL_freqdist = nltk.FreqDist(cvl_data_set)

cvl_list = [i for (i, _) in CVL_freqdist.most_common(50)]

org_data_set = []
for data in train_data:
    if data[2] == 'ORG':
        if re.search("\w+", data[1]):
            word = re.search("\w+", data[1]).group()
            if word[-1] in "은는이가을던의를에도들":
                org_data_set.append(word[:-1])
            else:
                org_data_set.append(word)
                
ORG_freqdist = nltk.FreqDist(org_data_set)

org_list = [i for (i, _) in ORG_freqdist.most_common(50)]
org_list

trm_data_set = []
for data in train_data:
    if data[2] == 'TRM':
        if re.search("\w+", data[1]):
            word = re.search("\w+", data[1]).group()
            if word[-1] in "은는이가을던의를에도들":
                trm_data_set.append(word[:-1])
            else:
                trm_data_set.append(word)
                
TRM_freqdist = nltk.FreqDist(trm_data_set)

trm_list = [i for (i, _) in TRM_freqdist.most_common(50)]

loc_data_set = []
for data in train_data:
    if data[2] == 'LOC':
        if re.search("\w+", data[1]):
            word = re.search("\w+", data[1]).group()
            #if word[-1] in "은는이가을던의를에도들":
            #    trm_data_set.append(word[:-1])
            #else:
            loc_data_set.append(word)
                
LOC_freqdist = nltk.FreqDist(loc_data_set)

loc_list = [i for (i, _) in LOC_freqdist.most_common(50)]


In [16]:
# feature extraction 영역

# 각 문장의 길이와 

# 고려할 만한 정보

# 맨 첫 문자와, 맨 마지막 문자를 사용할 수 있다.

# TIM, DAT 의 경우, 오전, 오후, 전반, 후반, 시, 분, 초 등이 존재할 수 있다.

#문자열에 숫자가 존재하는지 (isdigit() = True)을 통해 NUM을 판별해볼 수 있다.


# 가장 많이 사용하는 100개의 문자를 토대로 판별한다.
# freqdist = nltk.FreqDist(data[1] for data in train_data)
# common_list = [i[0] for i in freqdist.most_common(50)]

def feature_extraction(dataset, loc):
    features = {}
    
    #시간(DAT, TIM)을 위한 data
    for letter in "년월일시분초전후":
        features[f'has({letter})'] = letter in dataset[loc][1]
    
    features['word'] = dataset[loc][1]
        
    features['first_one'] = dataset[loc][1][0]
    features['first_two'] = dataset[loc][1][:2] if len(dataset[loc][1]) > 1 else dataset[loc][1]
    features['first_three'] = dataset[loc][1][:3] if len(dataset[loc][1]) > 2 else dataset[loc][1]
    
    features['last_one'] = dataset[loc][1][-1]
    features['last_two'] = dataset[loc][1][-2:]
    features['last_three'] = dataset[loc][1][-3:]
    
    features['delete_last_one'] = dataset[loc][1][:-1]
    features['delete_last_two'] = dataset[loc][1][:-2]
    features['delete_last_three'] = dataset[loc][1][:-3]
    
    features['before_word'] = '' if dataset[loc][0] == '1' else dataset[loc-1][1]
    features['after_word'] = '' if loc == len(dataset)-1 or dataset[loc+1][0] == '1' else dataset[loc+1][1]
    
    features['has_digit'] = any(letter.isdigit() for letter in dataset[loc][1])
    features['has_eng'] = True if re.search('[a-zA-Z]', dataset[loc][1]) else False

    return features


    #features['length'] = len(data[1])
    
    """
    features['in_cvl_list'] = False
    for i in cvl_list:
        if i in data[1]:
            features['in_cvl_list'] = True
    
    features['in_org_list'] = False
    for i in org_list:
        if i in data[1]:
            features['in_org_list'] = True
    
    features['in_trm_list'] = False
    for i in trm_list:
        if i in data[1]:
            features['in_trm_list'] = True
    
    features['in_loc_list'] = False
    for i in loc_list:
        if i in data[1]:
            features['in_loc_list'] = True
    """
    


In [17]:
# train, test_data의 모든 data에 feature_extraction을 적용하여 train, test_set 구현
train_set = [(feature_extraction(train_data, i), train_data[i][2])
             for i in range(len(train_data))]
test_set = [(feature_extraction(test_data, i), test_data[i][2])
             for i in range(len(test_data))]

# classifier을 만들고, test_set에 대해 accuracy를 측정한다.
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.8702424829349932

In [136]:
classifier.show_most_informative_features()

Most Informative Features
                  string = '.'                 - : AFW    =   6363.9 : 1.0
            first_letter = '1'               NUM : PER    =   4848.0 : 1.0
       delete_last_three = '2'               NUM : -      =   4507.3 : 1.0
              last_three = '.'                 - : AFW    =   4359.6 : 1.0
            first_letter = '2'               NUM : -      =   3917.3 : 1.0
       delete_last_three = '1'               NUM : -      =   3830.6 : 1.0
             last_letter = 'r'               TRM : -      =   3452.9 : 1.0
                last_two = '두'               NUM : -      =   3447.3 : 1.0
                last_two = '1일'              DAT : -      =   3335.5 : 1.0
            first_letter = '7'               DAT : -      =   3141.3 : 1.0


In [9]:
# error 가 자주 발생하는 어휘 판별
errors = []
for i in range(len(test_data)):
    guess = classifier.classify(feature_extraction(test_data, i))
    if guess != test_data[i][2]:
        errors.append((guess, test_data[i][2], test_data[i][1]))

In [10]:
errors
freq = nltk.FreqDist([data for data in errors])
freq
# len(errors)

FreqDist({('NUM', '-', '한'): 531, ('DAT', '-', '지난'): 130, ('DAT', '-', '전'): 118, ('NUM', 'DAT', '한'): 109, ('DAT', '-', '호시기'): 76, ('DAT', '-', '계절'): 70, ('DAT', '-', '유행기'): 70, ('DAT', '-', '몇'): 69, ('DAT', '-', '월경기'): 68, ('DAT', '-', '올'): 65, ...})

In [20]:
freqdist = nltk.FreqDist(data[2] for data in test_data)

In [21]:
freqdist.most_common()

[('-', 292164),
 ('NUM', 25966),
 ('CVL', 23972),
 ('PER', 19280),
 ('ORG', 18310),
 ('DAT', 13589),
 ('TRM', 8885),
 ('LOC', 8370),
 ('EVT', 7029),
 ('ANM', 2609),
 ('AFW', 2451),
 ('TIM', 1647),
 ('FLD', 949),
 ('MAT', 124),
 ('PLT', 87)]

In [22]:
for tag, _ in freqdist.most_common():
    data_div_set = []
    for data in test_set:
        if data[1] == tag:
            data_div_set.append(data)
    
    predict = nltk.classify.accuracy(classifier, data_div_set)
    print(tag, round(predict, 3))

- 0.878
NUM 0.916
CVL 0.881
PER 0.898
ORG 0.848
DAT 0.92
TRM 0.735
LOC 0.814
EVT 0.752
ANM 0.751
AFW 0.432
TIM 0.862
FLD 0.547
MAT 0.073
PLT 0.138


In [ ]:
# 빈도수를 바탕으로 cvl dataset 얻어내기
cvl_data_set = []
for data in train_data:
    if data[2] == 'CVL':
        if re.search("\w+", data[1]):
            word = re.search("\w+", data[1]).group()
            if word[-1] in "은는이가을던의를에도들":
                cvl_data_set.append(word[:-1])
            else:
                cvl_data_set.append(word)
                
CVL_freqdist = nltk.FreqDist(cvl_data_set)

cvl_list = [i for (i, _) in CVL_freqdist.most_common(50)]

# feature_extraction 함수 내부에서 아래와 같이 feature로 추가
features['in_cvl_list'] = False
for i in cvl_list:
    if i in dataset[loc][1]:
        features['in_cvl_list'] = True